In [1]:
import numpy as np
import pandas as pd

##### pivot
- 데이터 프레임의 컬럼에서 index, columns, values를 선택하여 데이터 프레임을 만드는 방법
- 아래 형태로 파라미터는 index, columns, values로 들어간다.
    - df.pivot(index, columns, values)
- index와 columns 데이터에 해당하는 values가 2개 이상이면 에러가 발생한다.

In [3]:
# titanic data load
# https://www.kaggle.com/c/titanic/data
# survived : 0-no, 1-yes
titanic = pd.read_csv('train.csv')
titanic.info()
titanic.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [4]:
# 성비와 객실등급에 따라 groupby를 하고 데이터의 수를 Counts 컬럼에 추가
titanic_f1 = pd.DataFrame(titanic, columns=["Pclass", "Sex"])
titanic_f1 = titanic.groupby(["Sex","Pclass"]).size().reset_index(name='Counts')
titanic_f1.tail()

,Sex,Pclass,Counts
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


In [5]:
# pivot : 객실등급과 남녀에 따른 데이터 수
# df.pivot(index, columns, values)
titanic_f1.pivot("Sex", "Pclass", "Counts")

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


In [6]:
# 생존과 객실등급에 따라 groupby를 하고 데이터의 수를 Counts 컬럼에 추가
titanic_f2 = pd.DataFrame(titanic, columns=["Pclass", "Survived"])
titanic_f2 = titanic.groupby(["Survived","Pclass"]).size().reset_index(name='Counts')
titanic_f2.tail()

,Survived,Pclass,Counts
1,0,2,97
2,0,3,372
3,1,1,136
4,1,2,87
5,1,3,119


In [7]:
# pivot : 객실등급과 생존에 따른 데이터 수
# df.pivot(index, columns, values)
titanic_f2.pivot("Pclass", "Survived", "Counts")

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


##### pivot_table
- pivot_table(values, index, columns, aggfunc)
    - values: value 데이터
    - index: 인덱스 리스트 데이터
    - columns: 컬럼 리스트 데이터
    - aggfunc: groupby aggregate 함수
    - fill_value : 데이터가 없을때 들어가는 데이터
    - dropna : 없는 데이터 컬럼은 제거

In [7]:
titanic_f3 = pd.DataFrame(titanic)
titanic_f3["Count"] = 1
titanic_f3.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Count
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,1
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,1


In [8]:
# pivot_table : 객실등급과 남녀에 따른 데이터 수
titanic_f3.pivot_table(values="Count", index=["Sex"], columns=["Pclass"], aggfunc=np.sum)

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


In [9]:
# pivot_table : 객실등급과 생존에 따른 데이터 수
titanic_f3.pivot_table(values="Count", index=["Pclass"], columns=["Survived"], aggfunc=np.sum)

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [10]:
# 객실등급과 성별별 생존 수
titanic_f3.pivot_table(values="Count", index=["Pclass","Sex"], columns=["Survived"], aggfunc=np.sum)

Survived         0   1
Pclass Sex            
1      female    3  91
       male     77  45
2      female    6  70
       male     91  17
3      female   72  72
       male    300  47

In [11]:
# 문제
# 아래와 같은 데이터 프레임을 pivot_table을 이용하여 남녀 성별별 생존 데이터를 구하시오.
df = titanic_f3.pivot_table(values="Count", index=["Survived"], columns=["Sex"], aggfunc=np.sum)
df

Sex,female,male
Survived,,
0,81,468
1,233,109


In [12]:
# total colum
df["total"] = df["female"] + df["male"] 
df

Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342


In [13]:
# total row
df.loc["total"] = df.loc[0] + df.loc[1]
df

Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342
total,314,577,891


In [14]:
# delete row 
df.drop("total", inplace=True)
df

Sex,female,male,total
Survived,,,
0,81,468,549
1,233,109,342


In [15]:
# delete column
df.drop("total", axis=1, inplace=True)
df

Sex,female,male
Survived,,
0,81,468
1,233,109


- SibSp : 형제/배우자
- Parch : 아이들

In [16]:
df = titanic_f3.pivot_table(values="Count", index=["Survived"], columns=["Parch","Pclass"], aggfunc=np.sum)
df

Parch        0                  1                 2                3       \
Pclass       1     2      3     1     2     3     1     2     3    2    3   
Survived                                                                    
0         64.0  86.0  295.0  10.0   8.0  35.0   5.0   3.0  32.0  NaN  2.0   
1         99.0  48.0   86.0  21.0  24.0  20.0  16.0  13.0  11.0  2.0  1.0   

Parch       4         5    6  
Pclass      1    3    3    3  
Survived                      
0         1.0  3.0  4.0  1.0  
1         NaN  NaN  1.0  NaN

In [17]:
# fill_value : 데이터가 없을때 들어가는 데이터
df = titanic_f3.pivot_table(values="Count", index=["Survived"], columns=["Parch","Pclass"],\
                            aggfunc=np.sum, fill_value=0)
df

Parch      0            1           2          3     4     5  6
Pclass     1   2    3   1   2   3   1   2   3  2  3  1  3  3  3
Survived                                                       
0         64  86  295  10   8  35   5   3  32  0  2  1  3  4  1
1         99  48   86  21  24  20  16  13  11  2  1  0  0  1  0

In [18]:
# dropna : 없는 데이터 컬럼은 제거
df = titanic_f3.pivot_table(values="Count", index=["Survived"], columns=["Parch","Pclass"],\
                            aggfunc=np.sum, fill_value=0, dropna=False)
df

Parch      0            1           2          3 ...     4        5        6  \
Pclass     1   2    3   1   2   3   1   2   3  1 ...  3  1  2  3  1  2  3  1   
Survived                                         ...                           
0         64  86  295  10   8  35   5   3  32  0 ...  2  1  0  3  0  0  4  0   
1         99  48   86  21  24  20  16  13  11  0 ...  1  0  0  0  0  0  1  0   

Parch           
Pclass    2  3  
Survived        
0         0  1  
1         0  0  

[2 rows x 21 columns]